## Requirements
- `astropy.io.fits` to access the fits files
- `matplotlib.pyplot` for plotting
- `numpy` for array handling
- `os` for file directory finding

In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import os
%matplotlib qt

# ONLY WORKS ON MY LAPTOP, COULDN'T GET THE PATH TO WORK FOR THE REMOTE REPO
direc = r"C:\Users\rober\OneDrive\Documents\_Docs\Year 4\TGP\TGP_Asteroids2_2025"

 Useful `.header` keys:
- RA
- DEC 
- AZIMUTH
- ALTITUDE
- FILTER 
- EXPOSURE

## Cropping
- open and display first image from night 1 to see the border, and crop until the border disappears

In [2]:
# READ FIRST IMAGE
image = fits.open(direc + r'\2025-10-09\PIRATE_165659_OSL_ROE_Asteroids_2_00_Standard_Star_1_00_Filter_R_00_2025_10_09_19_46_12.fits')[0].data

# REMOVE BORDER
image = image[21:2027,21:2027]

# PLOT
fig,ax = plt.subplots(figsize=(8,8))
ax.imshow(image, cmap='gray', vmax=1500)
plt.show()

- Crop all files from all nights, and all calibration frames

In [3]:
# EXTRACT ALL SCIENCE FRAMES
fnames_day1 = os.listdir(direc + r'\2025-10-09')
files_day1 = [direc + '\\2025-10-09\\' + file for file in fnames_day1]
fnames_day2 = os.listdir(direc + r'\2025-10-19')
files_day2 = [direc + '\\2025-10-19\\' + file for file in fnames_day2]
fnames_day3 = os.listdir(direc + r'\2025-10-22')
files_day3 = [direc + '\\2025-10-22\\' + file for file in fnames_day3]

# EXTRACT ALL CALIBRATION FRAMES
cnames_day1 = os.listdir(direc + r'\Calibrations_1')
calibs_day1 = [direc + '\\Calibrations_1\\' + file for file in cnames_day1]
cnames_day2 = os.listdir(direc + r'\Calibrations_2')
calibs_day2 = [direc + '\\Calibrations_2\\' + file for file in cnames_day2]
cnames_day3 = os.listdir(direc + r'\Calibrations_3')
calibs_day3 = [direc + '\\Calibrations_3\\' + file for file in cnames_day3]

# COMBINE ALL FRAMES
all_names = fnames_day1 + fnames_day2 + fnames_day3 + cnames_day1 + cnames_day2 + cnames_day3
all_files = files_day1 + files_day2 + files_day3 + calibs_day1 + calibs_day2 + calibs_day3

# CROP ALL AND GIVE KEY AS EXPOSURE CODE (E.G. 162431), SHAPES WILL BE (2006,2006)
cropped = {}
headers = {}
for (name, file) in zip(all_names, all_files):
    cropped[name[7:13]] = fits.open(file)[0].data[21:2027,21:2027]
    headers[name[7:13]] = fits.open(file)[0].header

# CHECK ALL CROPPED IMAGES HAVE THE CORRECT SHAPE
for key in cropped:
    if cropped[key].shape != (2006,2006):
        print(key)

## Bias subtraction

In [4]:
# GET MEDIAN BIAS FRAME FOR EACH NIGHT
bias_day1 = np.median(np.stack([cropped["164291"],
                                cropped["164292"],
                                cropped["164293"],
                                cropped["164294"]]), axis=0)
bias_day2 = np.median(np.stack([cropped["165895"],
                                cropped["165896"],
                                cropped["165897"],
                                cropped["165898"]]), axis=0)
bias_day3 = np.median(np.stack([cropped["166960"],
                                cropped["166961"],
                                cropped["166962"],
                                cropped["166963"]]), axis=0)

In [5]:
# BIAS-SUBTRACT THE DARK FRAMES
dark1_bs_60s = np.median(np.stack([cropped["164277"] - bias_day1,
                                   cropped["164278"] - bias_day1,
                                   cropped["164279"] - bias_day1,
                                   cropped["164280"] - bias_day1]), axis=0)
dark2_bs_60s = np.median(np.stack([cropped["165881"] - bias_day2,
                                   cropped["165882"] - bias_day2,
                                   cropped["165883"] - bias_day2,
                                   cropped["165884"] - bias_day2]), axis=0)
dark3_bs_60s = np.median(np.stack([cropped["166946"] - bias_day3,
                                   cropped["166947"] - bias_day3,
                                   cropped["166948"] - bias_day3,
                                   cropped["166949"] - bias_day3]), axis=0)

In [32]:
# EXTRACT ALL FLAT CODES
flat_codes1 = [str(i) for i in range(162424,162440) if i not in (162428,162429,162434,162435)] #4x BVR (12)
flat_codes1 += ["163459", "163461", "163462"] + [str(i) for i in range(164325, 164330)] # 8x I
flat_codes2 = [str(i) for i in range(165642,165658) if i not in (165646,165647,165652,165653)] #4x BVR
flat_codes3 = [str(i) for i in range(166095,166111) if i not in (166099,166100,166105,166106)] #4x BVR
print("codes extracted")

# BIAS SUBTRACT THE FLATS
flats1_bs = {}
for c in flat_codes1:
    flats1_bs[c] = cropped[c] - bias_day1
print("bias 1")
flats2_bs = {}
for c in flat_codes2:
    flats2_bs[c] = cropped[c] - bias_day2
print("bias 2")
flats3_bs = {}
for c in flat_codes3:
    flats3_bs[c] = cropped[c] - bias_day3
print("bias 3")

# SCALE DARK AND SUBTRACT
flats1_bs_ds = {}
for c in flat_codes1:
    flats1_bs_ds[c] = flats1_bs[c] - dark1_bs_60s * (headers[c]["EXPOSURE"]/60)
print("dark 1")
flats2_bs_ds = {}
for c in flat_codes2:
    flats2_bs_ds[c] = flats2_bs[c] - dark2_bs_60s * (headers[c]["EXPOSURE"]/60)
print("dark 2")
flats3_bs_ds = {}
for c in flat_codes3:
    flats3_bs_ds[c] = flats3_bs[c] - dark3_bs_60s * (headers[c]["EXPOSURE"]/60)
print("dark 3")

# STACK AND NORMALISE
flat_B1 = np.median(np.stack([flats1_bs_ds["162424"],
                              flats1_bs_ds["162425"],
                              flats1_bs_ds["162426"],
                              flats1_bs_ds["162427"]]), axis=0)
flat_B1 /= np.mean(flat_B1)
flat_V1 = np.median(np.stack([flats1_bs_ds["162430"],
                              flats1_bs_ds["162431"],
                              flats1_bs_ds["162432"],
                              flats1_bs_ds["162433"]]), axis=0)
flat_V1 /= np.mean(flat_V1)
flat_R1 = np.median(np.stack([flats1_bs_ds["162436"],
                              flats1_bs_ds["162437"],
                              flats1_bs_ds["162438"],
                              flats1_bs_ds["162439"]]), axis=0)
flat_R1 /= np.mean(flat_R1)
# flat_I1 ??
print("normalise 1")

flat_B2 = np.median(np.stack([flats2_bs_ds["165642"],
                              flats2_bs_ds["165643"],
                              flats2_bs_ds["165644"],
                              flats2_bs_ds["165645"]]), axis=0)
flat_B2 /= np.mean(flat_B2)
flat_V2 = np.median(np.stack([flats2_bs_ds["165648"],
                              flats2_bs_ds["165649"],
                              flats2_bs_ds["165650"],
                              flats2_bs_ds["165651"]]), axis=0)
flat_V2 /= np.mean(flat_V2)
flat_R2 = np.median(np.stack([flats2_bs_ds["165654"],
                              flats2_bs_ds["165655"],
                              flats2_bs_ds["165656"],
                              flats2_bs_ds["165657"]]), axis=0)
flat_R2 /= np.mean(flat_R2)
print("normalise 2")

flat_B3 = np.median(np.stack([flats3_bs_ds["166095"],
                              flats3_bs_ds["166096"],
                              flats3_bs_ds["166097"],
                              flats3_bs_ds["166098"]]), axis=0)
flat_B3 /= np.mean(flat_B3)
flat_V3 = np.median(np.stack([flats3_bs_ds["166101"],
                              flats3_bs_ds["166102"],
                              flats3_bs_ds["166103"],
                              flats3_bs_ds["166104"]]), axis=0)
flat_V3 /= np.mean(flat_V3)
flat_R3 = np.median(np.stack([flats3_bs_ds["166107"],
                              flats3_bs_ds["166108"],
                              flats3_bs_ds["166109"],
                              flats3_bs_ds["166110"]]), axis=0)
flat_R3 /= np.mean(flat_R3)
print("normalise 3")

codes extracted
bias 1
bias 2
bias 3
dark 1
dark 2
dark 3
normalise 1
normalise 2
normalise 3


In [ ]:
# PLOT ALL DAY 1 R, AND R1 FLAT
# focussed on big star, why still there
fig,axs = plt.subplots(2,3,figsize=(24,12))
flaxs = axs.flatten()
for a,c in zip(flaxs[:-2],flat_codes1[8:12]):
    a.imshow(flats1_bs_ds[c][750:850,1250:1350]/np.mean(flats1_bs_ds[c][750:850,1250:1350]),cmap="plasma",vmin=.9,vmax=1.1)
    a.set_title(c)
flaxs[-2].imshow(flat_R1, cmap="plasma", vmin=.9, vmax=1.1)
fig.tight_layout()
plt.show()
# (1200,700) to (1400,900)

In [58]:
# PLOT ALL DAY 1 B, AND B1 FLAT
fig,axs = plt.subplots(2,3,figsize=(24,12))
flaxs = axs.flatten()
for a,c in zip(flaxs[:-2],flat_codes1[:4]):
    a.imshow(flats1_bs_ds[c]/np.mean(flats1_bs_ds[c]),cmap="plasma",vmin=.94,vmax=1.06)
    a.set_title(c)
flaxs[-2].imshow(flat_B1, cmap="plasma", vmin=.94, vmax=1.06)
fig.tight_layout()
plt.show()

In [ ]:
# PLOT ALL R final flats
fig,axs = plt.subplots(1,3,figsize=(24,8))
axs[0].imshow(flat_R1, cmap="plasma", vmin=0.9, vmax=1.1)
axs[1].imshow(flat_R2, cmap="plasma", vmin=0.9, vmax=1.1)
axs[2].imshow(flat_R3, cmap="plasma", vmin=0.9, vmax=1.1)
fig.tight_layout(); plt.show()

In [59]:
#  PLOT ALL FLATS
fig,axs = plt.subplots(3,3,figsize=(12,12))
axs[0,0].imshow(flat_B1, cmap="plasma", vmin=.94, vmax=1.06)
axs[0,0].set_title("B1")
axs[0,1].imshow(flat_V1, cmap="plasma", vmin=.94, vmax=1.06)
axs[0,1].set_title("V1")
axs[0,2].imshow(flat_R1, cmap="plasma", vmin=.94, vmax=1.06)
axs[0,2].set_title("R1")
axs[1,0].imshow(flat_B2, cmap="plasma", vmin=.94, vmax=1.06)
axs[1,1].imshow(flat_V2, cmap="plasma", vmin=.94, vmax=1.06)
axs[1,2].imshow(flat_R2, cmap="plasma", vmin=.94, vmax=1.06)
axs[2,0].imshow(flat_B3, cmap="plasma", vmin=.94, vmax=1.06)
axs[2,1].imshow(flat_V3, cmap="plasma", vmin=.94, vmax=1.06)
axs[2,2].imshow(flat_R3, cmap="plasma", vmin=.94, vmax=1.06)

fig.tight_layout(); plt.show()